#  Orientation data - angle analysis 
Analysis of the angles in 2D, then possibly 3D.
Goal: find angular velocity to compare with event log. 

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
%matplotlib notebook

path = '../datasets/'
fname = 'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC_clean.h5'
rat_orientation = pd.read_hdf(path+fname, 'Orientation')

In [2]:
#dropping the y to go to 2D unitvector and then normalize - probably bad approach
i=20000

def create_vec(X, Y, Z, i):
    A = np.array([X[i],Y[i],Z[i]])
    return A

def create_flat_vec(X, Y, Z, i):
    A = np.array([X[i],Y[i],Z[i]])
    B = np.array([X[i],0   ,Z[i]])

    Bm = np.linalg.norm(A)
    B1 = B / Bm
    #print(B, B1)
    return B1

def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

X, Y, Z, Time = rat_orientation.X,rat_orientation.Y, rat_orientation.Z, rat_orientation.Time
dt = Time[i+1] - Time[i]

# XYZ vector angular speed
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)

CosTheta, Theta = angle_vec(V1, V2)
U = Theta/dt
print('XYZ',CosTheta, Theta)

# y -> to zero
XZ1 = create_flat_vec(X, Y, Z, i)
XZ2 = create_flat_vec(X, Y, Z, i+1)

CosTheta, Theta = angle_vec(XZ1, XZ2)
U = Theta/dt
print('XZ', CosTheta, Theta)

#y vector angular speed
Y1 = np.array([0, Y[i]  , 1])
Y2 = np.array([0, Y[i+1], 1])

CosTheta, ThetaY = angle_vec(Y1, Y2)
UY = ThetaY/dt
print('Y', CosTheta, ThetaY)


XYZ 0.999826053222 0.0186521752865
XZ 0.999993237954 0.00367751384604
Y 0.999896397674 0.0143947292729


In [3]:
for i in range(7000,7100):
    dt = Time[i+1] - Time[i]
    # XYZ vector angular speed
    V1 = create_vec(X, Y, Z, i)
    V2 = create_vec(X, Y, Z, i+1)

    CosTheta, Theta = angle_vec(V1, V2)
    U = Theta/dt
    #print('XYZ',Theta)
    #print('XYZ', CosTheta)

    # y -> to zero
    XZ1 = create_flat_vec(X, Y, Z, i)
    XZ2 = create_flat_vec(X, Y, Z, i+1)

    CosThetaXZ, ThetaXZ = angle_vec(XZ1, XZ2)
    U = ThetaXZ/dt

    #y vector angular speed
    Y1 = np.array([0, Y[i]  ,Z[i] ])
    Y2 = np.array([0, Y[i+1],Z[i+1]])

    CosThetaY, ThetaY = angle_vec(Y1, Y2)
    UY = ThetaY/dt
    #print('XZ', Theta, U, 'Y', ThetaY, UY)
    print('XZ', CosThetaXZ, 'Y', CosThetaY, 'Theta:XZ', ThetaXZ, 'Y', ThetaY)
    
    

XZ 0.999962345485 Y 0.99972509626 Theta:XZ 0.00867810478587 Y 0.0234485110762
XZ 0.999951004746 Y 0.999690542176 Theta:XZ 0.00989905592392 Y 0.02487865695
XZ 0.999937909547 Y 0.999573721473 Theta:XZ 0.0111437063772 Y 0.0291996169889
XZ 0.999962480488 Y 0.999661321865 Theta:XZ 0.00866253385168 Y 0.0260268036532
XZ 0.999925897544 Y 0.999090953808 Theta:XZ 0.0121740191342 Y 0.0426423252794
XZ 0.999854870913 Y 0.997447135156 Theta:XZ 0.0170371709673 Y 0.071469598721
XZ 0.999857564891 Y 0.996911670378 Theta:XZ 0.0168782991208 Y 0.078611965537
XZ 0.999859809995 Y 0.99573107425 Theta:XZ 0.0167447472662 Y 0.0924333976422
XZ 0.999834742003 Y 0.994371728768 Theta:XZ 0.0181803492021 Y 0.106146679115
XZ 0.999918489922 Y 0.995641332861 Theta:XZ 0.0127680213788 Y 0.0934006119255
XZ 0.999946937353 Y 0.999113678599 Theta:XZ 0.0103017587189 Y 0.0421058751185
XZ 0.999646568102 Y 0.974146439378 Theta:XZ 0.0265876933496 Y 0.227884778753
XZ 0.999870345909 Y 0.982696708181 Theta:XZ 0.016103222808 Y 0.186297

In [4]:
# rotation update
# thetaP = theta + dt /2 * omega * theta
# omega = thetaP - theta / (dt/2 * theta)

i = 40000

def omega(theta, thetaP, dt):
    return thetaP - theta / (dt/2 * theta)

dt = Time[i+1] - Time[i]
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)


# go to quaterions first, then check

print(dt)
print(omega(V1, V2, dt))

print(V2-V1)

0.004167
[-480.00794735 -480.66994191 -480.66595263]
[ 0.00279911  0.00015107 -0.00034176]


In [5]:
# get theta, go to quaternion, then extract the omega
i = 40000


def quat(theta, x, y, z):
    Q = np.array([    np.cos(theta/2),
                  x * np.sin(theta/2),
                  y * np.sin(theta/2),
                  z * np.sin(theta/2)])
    return Q

dt = Time[i+1] - Time[i]
V1 = create_vec(X, Y, Z, i)
V2 = create_vec(X, Y, Z, i+1)

CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V1))
Theta = np.arccos(CosTheta)
print(dt, Theta)


Q1 = quat(Theta, X[i], Y[i], Z[i])
Q2 = quat(Theta, X[i+1], Y[i+1], Z[i+1])

print(omega(Q1, Q2, 0.1))


0.004167 0.00282394086826
[-19.000001   -20.00006544 -20.00100015 -20.00099452]


## Second approach

In [6]:
def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

X, Y, Z, Time = rat_orientation.X,rat_orientation.Y, rat_orientation.Z, rat_orientation.Time
dt = Time[i+1] - Time[i]
i = 100

# get two vectors aligned with Y and Z axes
Vo = np.array([X[i], Y[i], Z[i]])  
Vz = np.array([X[i], Y[i], 0   ])  
Vy = np.array([X[i], 0   , Z[i]])
Vx = np.array([0   , Y[i], Z[i]])

# find the angles between them and orientation vector 
cosAlpha, alpha = angle_vec(Vo, Vz)
cosBeta , beta  = angle_vec(Vo, Vy)
cosGamma, gamma = angle_vec(Vo, Vx)

print(alpha, beta, gamma)

# cos -> get lenghts of decomposition
LenghtVz = cosAlpha
LenghtVy = cosBeta
LenghtVx = cosGamma

# find the tip point with flat Y
Vxy = np.array([LenghtVx, LenghtVz])
print(Vxy)


## ADDITIONAL FILTERING BY THE TOO BIG Y?

0.444837719905 0.142023711661 1.10067573692
[ 0.4529938   0.90268051]


In [7]:
def decompose_vec(X, Y, Z, i):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([X[i], Y[i], Z[i]])  
    Vz = np.array([X[i], Y[i], 0   ])  
    Vy = np.array([X[i], 0   , Z[i]])
    Vx = np.array([0   , Y[i], Z[i]])

    # find the angles between them and orientation vector 
    LenghtVz, alpha = angle_vec(Vo, Vz)
    LenghtVy, beta  = angle_vec(Vo, Vy)
    LenghtVx, gamma = angle_vec(Vo, Vx)
 
    # find the tip point with flat Y
    Vxy = np.array([LenghtVx, 0, LenghtVz])
    return Vxy, LenghtVy

# analyzie two separete angular velocities based on that 
i = 200

dt = Time[i+1] - Time[i]
Vxy1, Y1 = decompose_vec(X, Y, Z, i  )
Vxy2, Y2 = decompose_vec(X, Y, Z, i+1)

cosTheta, theta = angle_vec(Vxy1, Vxy2)
U = theta/dt
print(U)

UY1 = (Y1-Y2)/dt
print(UY1)



0.623716629169
0.215101265011


In [8]:
#function in the loop

for i in range(2000,2100):
    Vxy1, Y1 = decompose_vec(X, Y, Z, i  )
    Vxy2, Y2 = decompose_vec(X, Y, Z, i+1)

    cosTheta, theta = angle_vec(Vxy1, Vxy2)
    U   = theta / dt    
    UY1 = (Y1-Y2) / dt
    print(U, UY1)
    

1.33260296029 0.480353052281
1.5237065653 0.445576531409
1.52188958428 0.378276644655
1.66302082665 0.439302474934
1.870146619 0.435210188003
1.69032558548 0.415485157744
1.71839120529 0.539115758567
1.92323005776 0.588717792858
1.99176468004 0.665297005026
2.05715276173 0.691999365343
1.78878694199 0.293828760222
1.70562218099 0.685618630639
1.81802493439 0.607694782226
1.63206425018 0.554949533619
1.9834465704 0.766256514378
1.48407729107 0.53954862579
1.67442841527 0.695131466763
1.4730651044 0.6345625267
0.802372954257 0.561857651231
0.931559236964 0.578651501865
1.11535416798 0.572271921216
1.20141158421 0.556155836674
0.938002538843 0.50369758731
0.972105372639 0.419445386472
0.787764371635 0.547583965529
0.824478169422 0.331211085837
0.76144432328 0.539666028787
0.59161966631 0.338187299548
0.590827424233 0.475557990755
0.869495583165 0.680413786256
0.632260684954 0.633138109882
0.664032326582 0.70673217858
0.590202846447 0.387550487947
0.73383917056 0.512443995032
0.70764434386

## Moving problem to pandas df
Inside pandas update of cosTheta, theta, U, Uy

In [21]:
# loading data
dfOri = {}

fnameClean = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU_clean.h5',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC_clean.h5',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC_clean.h5', ]

path = '../datasets/'

for i, x in enumerate(fnameClean):
    dfOri[i] = pd.read_hdf(path+fnameClean[i], 'Orientation')
    

In [22]:
def angle_vec(V1, V2):
    CosTheta = np.dot(V1,V2) / (np.linalg.norm(V1) * np.linalg.norm(V2))
    Theta = np.arccos(CosTheta)
    return CosTheta, Theta

def decompose_vec(X, Y, Z):
    # get two vectors aligned with Y and Z axes
    Vo = np.array([X, Y, Z])  
    Vz = np.array([X, Y, 0])  
    Vy = np.array([X, 0, Z])
    Vx = np.array([0, Y, Z])

    # find the angles between them and orientation vector 
    LenghtVz, alpha = angle_vec(Vo, Vz)
    LenghtVy, beta  = angle_vec(Vo, Vy)
    LenghtVx, gamma = angle_vec(Vo, Vx)
 
    return ([LenghtVx, 0, LenghtVz])

dfV = {}

# extraction of positon and time changes
for i, x in enumerate(fnameClean):
    dfV[i] = dfOri[i].diff()#.dropna()
    dfOri[i]['dT'] = dfV[i].Time
    
    
def compute_angle(df):
    next_df = df.shift(-1)
    return angle_vec(df.lenght, next_df.lenght)
    
i = 1

df = dfOri[2]
Vxz  = decompose_vec(df.X[i]  , df.Y[i]  , df.Z[i])
Vxz1 = decompose_vec(df.X[i+1], df.Y[i+1], df.Z[i+1])
cosTheta, theta = angle_vec(Vxz, Vxz1)
U = theta/df.dT[i]
print(U)

dfOri[1]['lenght'] = dfOri[1].apply(lambda row: decompose_vec(row.X, row.Y, row.Z) , axis=1)
#dfOri[1]['theta'] = compute_angle(dfOri[1])
#print(dfOri[1].lenght)
print(dfOri[1].dT)
                                   
    
#for i, x in enumerate(fnameClean):
#    dfOri[i]['lenght'] = dfOri[i].apply(lambda row:decompose_vec(row.X, row.Y, row.Z) , axis=1)


0.231864422402
3665           NaN
3666      0.004167
3668      0.008333
3669      0.004167
3670      0.004167
3671      0.004166
3672      0.004167
3673      0.004167
3674      0.004166
3675      0.004167
3676      0.004167
3677      0.004166
3678      0.004167
3679      0.004167
3680      0.004166
3681      0.004167
3682      0.004167
3683      0.004166
3684      0.004167
3685      0.004167
3686      0.004166
3687      0.004167
3688      0.004167
3689      0.004166
3690      0.004167
3691      0.004167
3692      0.004166
3693      0.004167
3694      0.004167
3695      0.004166
            ...   
158629    0.004167
158630    0.004166
158631    0.004167
158632    0.004167
158633    0.004166
158634    0.004167
158635    0.004167
158636    0.004166
158637    0.004167
158638    0.004167
158639    0.004166
158640    0.004167
158641    0.004167
158642    0.004166
158643    0.004167
158644    0.004167
158645    0.004166
158646    0.004167
158647    0.004167
158648    0.004166
158649    0.0041